In [8]:
# NYC Weather Data via Meteostat (using NOAA data)
import requests
import pandas as pd

token = "ozFjosloVwEurlSqyGOZLcvRZGtAEFWm"

url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"

params = {
    "datasetid": "GHCND",
    "stationid": "GHCND:USW00094728",
    "startdate": "2023-01-01",
    "enddate": "2023-01-31",
    "limit": 1000,
    "units": "metric"
}

headers = {
    "token": token
}

res = requests.get(url, params=params, headers=headers)
data = res.json()

# Преобразуем в DataFrame
records = data.get("results", [])
df = pd.DataFrame(records)
print(df.head())

                  date datatype            station attributes    value
0  2023-01-01T00:00:00     ADPT  GHCND:USW00094728       ,,W,     39.0
1  2023-01-01T00:00:00     ASLP  GHCND:USW00094728       ,,W,  10125.0
2  2023-01-01T00:00:00     ASTP  GHCND:USW00094728       ,,W,  10071.0
3  2023-01-01T00:00:00     AWBT  GHCND:USW00094728       ,,W,     78.0
4  2023-01-01T00:00:00     AWND  GHCND:USW00094728       ,,W,      2.2


In [9]:
# Отфильтруем нужные показатели
needed_codes = ["TMAX", "TMIN", "PRCP"]

df_filtered = df[df["datatype"].isin(needed_codes)].copy()

# Значения делим на 10
df_filtered["value"] = df_filtered["value"] / 10

# Сводим в табличку с колонками
pivot = df_filtered.pivot_table(
    index="date",
    columns="datatype",
    values="value"
)

print(pivot.head())

datatype             PRCP  TMAX  TMIN
date                                 
2023-01-01T00:00:00  0.00  1.28  0.94
2023-01-02T00:00:00  0.05  1.33  0.94
2023-01-03T00:00:00  1.07  1.44  0.83
2023-01-04T00:00:00  0.05  1.89  0.94
2023-01-05T00:00:00  0.03  1.00  0.67


In [4]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# NOAA API токен
token = "ozFjosloVwEurlSqyGOZLcvRZGtAEFWm"

# URL и настройки
url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"
station_id = "GHCND:USW00094728"  # NYC Central Park

headers = {"token": token}

# Генерация месяцев 2025 года
def get_month_ranges(year):
    ranges = []
    for month in range(1, 13):
        start = datetime(year, month, 1)
        if month == 12:
            end = datetime(year + 1, 1, 1) - timedelta(days=1)
        else:
            end = datetime(year, month + 1, 1) - timedelta(days=1)
        ranges.append((start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")))
    return ranges

# Скачиваем по каждому месяцу
all_data = []

for start_date, end_date in get_month_ranges(2025):
    print(f"📅 Загружаем данные: {start_date} — {end_date}")
    params = {
        "datasetid": "GHCND",
        "stationid": station_id,
        "startdate": start_date,
        "enddate": end_date,
        "limit": 1000,
        "units": "metric"
    }

    res = requests.get(url, params=params, headers=headers)
    
    if res.status_code != 200:
        print(f"⚠️ Ошибка при загрузке {start_date} — {end_date}: {res.status_code}")
        continue

    records = res.json().get("results", [])
    all_data.extend(records)

# В DataFrame
df = pd.DataFrame(all_data)

# Преобразуем дату в datetime
df["date"] = pd.to_datetime(df["date"])



# 1. Убедимся, что дата в нужном формате
df['date'] = pd.to_datetime(df['date'])

# 2. Разворачиваем таблицу: каждая дата и станция — одна строка
df_wide = df.pivot_table(
    index=['date', 'station'],
    columns='datatype',
    values='value',
    aggfunc='first'  # если вдруг есть дубликаты
).reset_index()

# 3. Опционально: убрать имя оси столбцов (datatype)
df_wide.columns.name = None

# 4. Смотрим результат
print(df_wide.head())

# Сохраняем в CSV
df_wide.to_csv("nyc_weather_2025.csv", index=False)

print("✅ Файл сохранен!")

📅 Загружаем данные: 2025-01-01 — 2025-01-31
📅 Загружаем данные: 2025-02-01 — 2025-02-28
📅 Загружаем данные: 2025-03-01 — 2025-03-31
📅 Загружаем данные: 2025-04-01 — 2025-04-30
📅 Загружаем данные: 2025-05-01 — 2025-05-31
📅 Загружаем данные: 2025-06-01 — 2025-06-30
📅 Загружаем данные: 2025-07-01 — 2025-07-31
📅 Загружаем данные: 2025-08-01 — 2025-08-31
📅 Загружаем данные: 2025-09-01 — 2025-09-30
📅 Загружаем данные: 2025-10-01 — 2025-10-31
📅 Загружаем данные: 2025-11-01 — 2025-11-30
📅 Загружаем данные: 2025-12-01 — 2025-12-31
        date            station  AWND    PGTM  PRCP  SNOW  SNWD  TMAX  TMIN  \
0 2025-01-01  GHCND:USW00094728   3.7  1606.0   0.0   0.0   0.0  10.6   3.9   
1 2025-01-02  GHCND:USW00094728   4.3  1508.0   0.0   0.0   0.0   6.1   0.6   
2 2025-01-03  GHCND:USW00094728   3.5  1439.0   0.0   0.0   0.0   3.9  -0.5   
3 2025-01-04  GHCND:USW00094728   4.5  1302.0   0.0   0.0   0.0   0.6  -2.1   
4 2025-01-05  GHCND:USW00094728   3.8  1614.0   0.0   0.0   0.0   0.6  -2.1  

In [6]:
df_wide

,date,station,AWND,PGTM,PRCP,SNOW,SNWD,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT06,WT08
0,2025-01-01,GHCND:USW00094728,3.7,1606.0,0.0,0.0,0.0,10.6,3.9,280.0,270.0,9.8,16.1,NaN,NaN,NaN,NaN,NaN
1,2025-01-02,GHCND:USW00094728,4.3,1508.0,0.0,0.0,0.0,6.1,0.6,280.0,230.0,8.9,17.4,NaN,NaN,NaN,NaN,NaN
2,2025-01-03,GHCND:USW00094728,3.5,1439.0,0.0,0.0,0.0,3.9,-0.5,230.0,230.0,6.7,12.1,NaN,NaN,NaN,NaN,NaN
3,2025-01-04,GHCND:USW00094728,4.5,1302.0,0.0,0.0,0.0,0.6,-2.1,270.0,320.0,9.8,17.9,NaN,NaN,NaN,NaN,NaN
4,2025-01-05,GHCND:USW00094728,3.8,1614.0,0.0,0.0,0.0,0.6,-2.1,310.0,310.0,8.1,13.4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,2025-07-08,GHCND:USW00094728,1.4,NaN,2.3,0.0,0.0,33.9,22.8,220.0,210.0,6.3,12.1,1.0,NaN,1.0,NaN,NaN
189,2025-07-09,GHCND:USW00094728,1.0,NaN,1.3,0.0,0.0,32.2,22.8,240.0,240.0,4.5,8.1,1.0,NaN,NaN,NaN,NaN
190,2025-07-10,GHCND:USW00094728,0.8,NaN,4.6,0.0,0.0,28.9,21.1,60.0,240.0,3.6,5.4,1.0,NaN,NaN,NaN,NaN
191,2025-07-11,GHCND:USW00094728,1.1,NaN,0.0,0.0,0.0,28.9,22.8,130.0,130.0,4.0,7.2,NaN,NaN,NaN,NaN,NaN


In [12]:
%pip install pyarrow==15.0.2
import pyarrow
import pandas as pd




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 85.0 kB/s eta 0:00:0000:0100:10
Note: you may need to restart the kernel to use updated packages.


NameError: name 'df1' is not defined

In [13]:
# Путь к файлу (поменяй на нужный)
file_path = 'data/nyc/green_tripdata_2025-01.parquet'

# Чтение parquet-файла
df1 = pd.read_parquet(file_path)

# Посмотреть первые строки
print(df1.head())

# Посмотреть список всех колонок
print(df1.columns)

   VendorID lpep_pickup_datetime lpep_dropoff_datetime store_and_fwd_flag  \
0         2  2025-01-01 00:03:01   2025-01-01 00:17:12                  N   
1         2  2025-01-01 00:19:59   2025-01-01 00:25:52                  N   
2         2  2025-01-01 00:05:29   2025-01-01 00:07:21                  N   
3         2  2025-01-01 00:52:24   2025-01-01 01:07:52                  N   
4         2  2025-01-01 00:25:05   2025-01-01 01:01:10                  N   

   RatecodeID  PULocationID  DOLocationID  passenger_count  trip_distance  \
0         1.0            75           235              1.0           5.93   
1         1.0           166            75              1.0           1.32   
2         5.0           171            73              1.0           0.41   
3         1.0            74           223              1.0           4.12   
4         1.0            66           158              1.0           4.71   

   fare_amount  ...  mta_tax  tip_amount  tolls_amount  ehail_fee  \
0    

In [15]:
import os
# Путь к папке с parquet-файлами
folder = "Data/nyc/"

# Найти все parquet-файлы
files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith('.parquet')]

print(f"Найдено файлов: {len(files)}")

# Чтение всех файлов и склейка
frames = []
for file in files:
    try:
        print(f"Читаю файл: {file}")
        df = pd.read_parquet(file, engine='pyarrow')
        frames.append(df)
    except Exception as e:
        print(f"❌ Ошибка при чтении файла {file}: {e}")

# Объединяем в один DataFrame
if frames:
    df_all = pd.concat(frames, ignore_index=True)
    print(f"✅ Всего строк в объединенном датасете: {len(df_all)}")
else:
    df_all = pd.DataFrame()
    print("⚠️ Нет данных для склейки.")

# Сохраняем в один parquet (или CSV)
output_file = os.path.join(folder, "green_tripdata_merged.parquet")
df_all.to_parquet(output_file, engine='pyarrow', index=False)
print(f"✅ Данные сохранены в {output_file}")

# Если хочешь сохранить в CSV:
# df_all.to_csv(os.path.join(folder, "green_tripdata_merged.csv"), index=False)

Найдено файлов: 16
Читаю файл: Data/nyc/fhv_tripdata_2025-02.parquet
Читаю файл: Data/nyc/fhvhv_tripdata_2025-04.parquet
Читаю файл: Data/nyc/yellow_tripdata_2025-02.parquet
Читаю файл: Data/nyc/yellow_tripdata_2025-03.parquet
Читаю файл: Data/nyc/green_tripdata_2022-01.parquet
Читаю файл: Data/nyc/green_tripdata_2025-03.parquet
Читаю файл: Data/nyc/fhv_tripdata_2025-03.parquet
Читаю файл: Data/nyc/yellow_tripdata_2022-01.parquet
Читаю файл: Data/nyc/yellow_tripdata_2025-01.parquet
Читаю файл: Data/nyc/green_tripdata_2025-01.parquet
Читаю файл: Data/nyc/fhv_tripdata_2025-01.parquet
Читаю файл: Data/nyc/green_tripdata_2022-02.parquet
Читаю файл: Data/nyc/fhvhv_tripdata_2025-03.parquet
Читаю файл: Data/nyc/green_tripdata_2025-04.parquet
Читаю файл: Data/nyc/fhv_tripdata_2025-04.parquet
Читаю файл: Data/nyc/yellow_tripdata_2025-04.parquet
✅ Всего строк в объединенном датасете: 65566563
✅ Данные сохранены в Data/nyc/green_tripdata_merged.parquet


In [17]:
df_all.to_csv(os.path.join(folder, "green_tripdata_merged.csv"), index=False)

KeyboardInterrupt: 

In [18]:
df_all.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,hvfhs_license_num,originating_base_num,request_datetime,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Airport_fee,lpep_pickup_datetime,lpep_dropoff_datetime,ehail_fee,trip_type
0,B00014,2025-02-01 00:00:00,2025-02-01 00:33:00,NaN,NaN,NaN,B00014,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
1,B00053,2025-02-01 00:24:00,2025-02-01 00:39:00,NaN,NaN,NaN,B00053,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
2,B00112,2025-02-01 00:15:13,2025-02-01 00:22:34,NaN,14.0,NaN,B00112,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
3,B00112,2025-02-01 00:28:02,2025-02-01 00:35:54,NaN,14.0,NaN,B00112,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN
4,B00221,2025-02-01 00:12:26,2025-02-01 00:18:11,NaN,167.0,NaN,B00221,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN


In [20]:
df_all.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number',
       'hvfhs_license_num', 'originating_base_num', 'request_datetime',
       'on_scene_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID',
       'trip_miles', 'trip_time', 'base_passenger_fare', 'tolls', 'bcf',
       'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips',
       'driver_pay', 'shared_request_flag', 'shared_match_flag',
       'access_a_ride_flag', 'wav_request_flag', 'wav_match_flag',
       'cbd_congestion_fee', 'VendorID', 'tpep_pickup_datetime',
       'tpep_dropoff_datetime', 'passenger_count', 'trip_distance',
       'RatecodeID', 'store_and_fwd_flag', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount', 'Airport_fee',
       'lpep_pickup_datetime', 'lpep_dropoff_datetime', 'ehail_fee',
       'trip_type'],
      dtype='

In [3]:
import pandas as pd

# 1. Прочитать датасет
df = pd.read_parquet("data/nyc/green_tripdata_merged.parquet")
print("Форма до изменений:", df.shape)

# 2. Создаём столбец pickup_datetime
pickup_cols = [
    "pickup_datetime",
    "tpep_pickup_datetime",
    "lpep_pickup_datetime"
]

# Найдём первый непустой pickup_datetime для каждой строки
# Если колонка отсутствует в датафрейме — не берём её
existing_cols = [col for col in pickup_cols if col in df.columns]

if not existing_cols:
    raise Exception("Нет колонок начала поездки в датасете!")

df["pickup_datetime"] = df[existing_cols].bfill(axis=1).iloc[:, 0]

# Приведём к datetime
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"], errors='coerce')

# 3. Удаляем все колонки конца поездки
drop_cols = [
    "dropOff_datetime",
    "tpep_dropoff_datetime",
    "lpep_dropoff_datetime",
    "dropoff_datetime"
]
drop_cols = [col for col in drop_cols if col in df.columns]

df.drop(columns=drop_cols, inplace=True)

# 4. Создаём колонку с часом поездки
df["pickup_hour"] = df["pickup_datetime"].dt.hour

# 5. Проверим результат
print("Форма после изменений:", df.shape)
print(df.head())

# 6. Сохраним результат
df.to_parquet("green_tripdata_cleaned.parquet", index=False)
print("✅ Датасет сохранён в green_tripdata_cleaned.parquet!")

Форма до изменений: (65566563, 50)
Форма после изменений: (65566563, 47)
  dispatching_base_num     pickup_datetime  PUlocationID  DOlocationID  \
0               B00014 2025-02-01 00:00:00           NaN           NaN   
1               B00053 2025-02-01 00:24:00           NaN           NaN   
2               B00112 2025-02-01 00:15:13           NaN          14.0   
3               B00112 2025-02-01 00:28:02           NaN          14.0   
4               B00221 2025-02-01 00:12:26           NaN         167.0   

   SR_Flag Affiliated_base_number hvfhs_license_num originating_base_num  \
0      NaN                 B00014              None                 None   
1      NaN                 B00053              None                 None   
2      NaN                 B00112              None                 None   
3      NaN                 B00112              None                 None   
4      NaN                 B00221              None                 None   

  request_datetime on_sce

In [3]:
import pandas as pd

# путь к твоему parquet-файлу
file_path = "Data/nyc/green_tripdata_cleaned.parquet"

# читаем датасет
df = pd.read_parquet(file_path)

# список колонок для удаления
cols_to_drop = [
    "dispatching_base_num", "originating_base_num", "Affiliated_base_number",
    "hvfhs_license_num", "request_datetime", "on_scene_datetime",
    "dropOff_datetime", "trip_miles", "shared_request_flag",
    "shared_match_flag", "access_a_ride_flag", "wav_request_flag",
    "wav_match_flag", "SR_Flag", "PGTM", "WDF2", "WDF5", "WSF2", "WSF5"
]

# удаляем, если такие колонки существуют
df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

print("✅ Ненужные колонки удалены!")
print(df.head())

# сохраним обратно в parquet или csv
df.to_parquet("Data/nyc/green_tripdata_cleaned.parquet", index=False)

KeyboardInterrupt: 

In [2]:
df.shape

(65566563, 32)

In [1]:
import pandas as pd

# загружаем твой датасет
file = 'Data/nyc/green_tripdata_cleaned.parquet'

df = pd.read_parquet(file)

# создаем новый столбец pickup_location_id:
# - сначала берем PULocationID, если он не пустой
# - если пустой → берем PUlocationID
df["pickup_location_id"] = df["PULocationID"].fillna(df["PUlocationID"])

# то же самое для dropoff_location_id
df["dropoff_location_id"] = df["DOLocationID"].fillna(df["DOlocationID"])

# убираем исходные 4 колонки
df.drop(columns=["PULocationID", "PUlocationID", "DOLocationID", "DOlocationID"], inplace=True)

df

,pickup_datetime,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,...,tolls_amount,improvement_surcharge,total_amount,Airport_fee,lpep_pickup_datetime,ehail_fee,trip_type,pickup_hour,pickup_location_id,dropoff_location_id
0,2025-02-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,None,NaN,0,NaN,NaN
1,2025-02-01 00:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,None,NaN,0,NaN,NaN
2,2025-02-01 00:15:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,None,NaN,0,NaN,14.0
3,2025-02-01 00:28:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,None,NaN,0,NaN,14.0
4,2025-02-01 00:12:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,None,NaN,0,NaN,167.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65566558,2025-04-30 23:37:25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,5.07,NaN,NaT,None,NaN,23,48.0,243.0
65566559,2025-04-30 23:53:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,25.75,NaN,NaT,None,NaN,23,234.0,142.0
65566560,2025-04-30 23:31:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,3.81,NaN,NaT,None,NaN,23,68.0,45.0
65566561,2025-04-30 23:27:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,25.16,NaN,NaT,None,NaN,23,186.0,141.0


In [3]:
df.to_parquet("Data/nyc/green_tripdata_cleaned.parquet", index=False)

In [4]:
df

,pickup_datetime,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,...,tolls_amount,improvement_surcharge,total_amount,Airport_fee,lpep_pickup_datetime,ehail_fee,trip_type,pickup_hour,pickup_location_id,dropoff_location_id
0,2025-02-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,None,NaN,0,NaN,NaN
1,2025-02-01 00:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,None,NaN,0,NaN,NaN
2,2025-02-01 00:15:13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,None,NaN,0,NaN,14.0
3,2025-02-01 00:28:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,None,NaN,0,NaN,14.0
4,2025-02-01 00:12:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaT,None,NaN,0,NaN,167.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65566558,2025-04-30 23:37:25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,5.07,NaN,NaT,None,NaN,23,48.0,243.0
65566559,2025-04-30 23:53:37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,25.75,NaN,NaT,None,NaN,23,234.0,142.0
65566560,2025-04-30 23:31:55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,3.81,NaN,NaT,None,NaN,23,68.0,45.0
65566561,2025-04-30 23:27:36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,25.16,NaN,NaT,None,NaN,23,186.0,141.0


In [7]:
# Преобразуем pickup_datetime в datetime
df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])

# Извлекаем дату и час
df["pickup_date"] = df["pickup_datetime"].dt.date
df["pickup_hour"] = df["pickup_datetime"].dt.hour

# Группирующие ключи
group_cols = ["pickup_location_id", "pickup_date", "pickup_hour"]

# Оставляем только числовые колонки
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()

# Исключаем ключи из агрегируемых колонок
numeric_cols = [c for c in numeric_cols if c not in group_cols]

# Формируем словарь агрегаций
agg_dict = {col: "mean" for col in numeric_cols}

# Добавим count поездок
agg_dict["pickup_datetime"] = "count"

# Агрегируем
agg_df = (
    df
    .groupby(group_cols)
    .agg(agg_dict)
    .reset_index()
)

# Переименуем колонку count
agg_df = agg_df.rename(columns={"pickup_datetime": "ride_count"})

print("✅ Итоговый агрегированный DataFrame:")
print(agg_df.head())

# Сохраняем результат
agg_df.to_csv("green_tripdata_aggregated.csv", index=False)
print("✅ Файл green_tripdata_aggregated.csv сохранен.")

✅ Итоговый агрегированный DataFrame:
   pickup_location_id pickup_date  pickup_hour  trip_time  \
0                 1.0  2022-01-01            4        NaN   
1                 1.0  2022-01-01            5        NaN   
2                 1.0  2022-01-01            7        NaN   
3                 1.0  2022-01-01           10        NaN   
4                 1.0  2022-01-01           11        NaN   

   base_passenger_fare  tolls  bcf  sales_tax  congestion_surcharge  \
0                  NaN    NaN  NaN        NaN                   0.0   
1                  NaN    NaN  NaN        NaN                   0.0   
2                  NaN    NaN  NaN        NaN                   0.0   
3                  NaN    NaN  NaN        NaN                   0.0   
4                  NaN    NaN  NaN        NaN                   0.0   

   airport_fee  ...  extra  mta_tax  tip_amount  tolls_amount  \
0          0.0  ...    0.0      0.0        0.00          0.00   
1          0.0  ...    0.0      0.0    

In [15]:
agg_df.describe()

,pickup_location_id,pickup_hour,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,Airport_fee,trip_type,dropoff_location_id,ride_count
count,731727.000000,731727.000000,368645.000000,368645.000000,368645.000000,368645.000000,368645.000000,611287.000000,432817.000000,368645.000000,...,539524.000000,539524.000000,539524.000000,539524.000000,539524.000000,539524.000000,304528.000000,76810.000000,730909.000000,731727.000000
mean,135.980493,11.852963,1075.012252,25.034043,1.012104,0.625903,2.117467,0.898329,0.149796,0.929753,...,0.524170,0.472277,1.678624,0.708208,0.771031,27.439769,0.041589,1.091875,141.289651,81.282772
std,76.386939,6.690104,286.764673,9.213515,1.388191,0.236062,0.725290,1.053914,0.327768,1.012837,...,0.880901,0.124744,3.002394,1.820118,0.327294,20.573412,0.246830,0.268890,40.554882,135.051656
min,1.000000,0.000000,1.000000,-5.910000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-3.250000,-0.250000,0.000000,0.000000,0.000000,-39.500000,0.000000,1.000000,1.000000,1.000000
25%,70.000000,6.000000,906.917241,19.448687,0.284296,0.484833,1.698049,0.000000,0.000000,0.295297,...,0.000000,0.487179,0.000000,0.000000,0.500000,18.105000,0.000000,1.000000,120.036585,4.000000
50%,138.000000,12.000000,1043.865385,22.678113,0.648000,0.566026,1.970548,0.248418,0.029297,0.675278,...,0.000000,0.500000,0.838750,0.000000,0.984962,22.956181,0.000000,1.000000,144.785714,26.000000
75%,204.000000,18.000000,1197.365297,28.106582,1.248150,0.701104,2.363333,2.329852,0.148810,1.257387,...,0.793919,0.500000,2.482950,0.301739,1.000000,31.570000,0.000000,1.000000,163.449102,105.000000
max,265.000000,23.000000,7489.500000,415.220000,39.730000,10.455000,37.125000,2.750000,5.000000,50.000000,...,11.000000,10.666667,250.000000,101.780000,1.000000,5155.930385,4.250000,2.000000,265.000000,2289.000000


In [16]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# NOAA API токен
token = "ozFjosloVwEurlSqyGOZLcvRZGtAEFWm"

headers = {"token": token}

# --- Получаем список станций в NYC ---
url_stations = "https://www.ncei.noaa.gov/cdo-web/api/v2/stations"
params_stations = {
    "datasetid": "GHCND",
    "extent": "40.5,-74.3,40.9,-73.7",  # рамка вокруг NYC
    "limit": 1000,
}
res = requests.get(url_stations, params=params_stations, headers=headers)
stations = res.json().get("results", [])

df_stations = pd.DataFrame(stations)
print("✅ Найдено станций:", len(df_stations))
print(df_stations[["id", "name"]])

# --- Функция дат ---
def get_month_ranges(year):
    ranges = []
    for month in range(1, 13):
        start = datetime(year, month, 1)
        if month == 12:
            end = datetime(year + 1, 1, 1) - timedelta(days=1)
        else:
            end = datetime(year, month + 1, 1) - timedelta(days=1)
        ranges.append((start.strftime("%Y-%m-%d"), end.strftime("%Y-%m-%d")))
    return ranges

# --- Скачиваем данные для всех станций ---
url_data = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"

all_data = []

for station_id in df_stations["id"]:
    for start_date, end_date in get_month_ranges(2025):
        print(f"🔎 Скачиваем {station_id}: {start_date} — {end_date}")
        
        params_data = {
            "datasetid": "GHCND",
            "stationid": station_id,
            "startdate": start_date,
            "enddate": end_date,
            "limit": 1000,
            "units": "metric"
        }
        
        res = requests.get(url_data, params=params_data, headers=headers)
        
        if res.status_code != 200:
            print(f"⚠️ Ошибка {res.status_code} для станции {station_id} ({start_date} — {end_date})")
            continue
        
        records = res.json().get("results", [])
        all_data.extend(records)

# --- Преобразуем в таблицу ---
df = pd.DataFrame(all_data)
print("✅ Загружено строк:", len(df))

if not df.empty:
    df["date"] = pd.to_datetime(df["date"])

    df_wide = df.pivot_table(
        index=["date", "station"],
        columns="datatype",
        values="value",
        aggfunc="first"
    ).reset_index()

    df_wide.columns.name = None
    df_wide.to_csv("nyc_weather_allstations_2025.csv", index=False)
    print("✅ Данные сохранены в nyc_weather_allstations_2025.csv")

else:
    print("⚠️ Данных нет.")
    

✅ Найдено станций: 99
                   id                                 name
0   GHCND:US1NJBG0011        NORTH ARLINGTON 0.7 NE, NJ US
1   GHCND:US1NJBG0012             WOOD RIDGE 0.6 SE, NJ US
2   GHCND:US1NJBG0013              RUTHERFORD 1.2 N, NJ US
3   GHCND:US1NJBG0015       NORTH ARLINGTON 0.7 WNW, NJ US
4   GHCND:US1NJBG0018        PALISADES PARK 0.2 WNW, NJ US
..                ...                                  ...
94  GHCND:USW00014786  NEW YORK FLOYD BENNETT FIELD, NY US
95  GHCND:USW00054743     CALDWELL ESSEX CO AIRPORT, NJ US
96  GHCND:USW00094728          NY CITY CENTRAL PARK, NY US
97  GHCND:USW00094741             TETERBORO AIRPORT, NJ US
98  GHCND:USW00094789     JFK INTERNATIONAL AIRPORT, NY US

[99 rows x 2 columns]
🔎 Скачиваем GHCND:US1NJBG0011: 2025-01-01 — 2025-01-31
⚠️ Ошибка 503 для станции GHCND:US1NJBG0011 (2025-01-01 — 2025-01-31)
🔎 Скачиваем GHCND:US1NJBG0011: 2025-02-01 — 2025-02-28
🔎 Скачиваем GHCND:US1NJBG0011: 2025-03-01 — 2025-03-31
🔎 Скачиваем 

🔎 Скачиваем GHCND:US1NJES0011: 2025-07-01 — 2025-07-31
🔎 Скачиваем GHCND:US1NJES0011: 2025-08-01 — 2025-08-31
🔎 Скачиваем GHCND:US1NJES0011: 2025-09-01 — 2025-09-30
🔎 Скачиваем GHCND:US1NJES0011: 2025-10-01 — 2025-10-31
🔎 Скачиваем GHCND:US1NJES0011: 2025-11-01 — 2025-11-30
🔎 Скачиваем GHCND:US1NJES0011: 2025-12-01 — 2025-12-31
🔎 Скачиваем GHCND:US1NJES0015: 2025-01-01 — 2025-01-31
🔎 Скачиваем GHCND:US1NJES0015: 2025-02-01 — 2025-02-28
🔎 Скачиваем GHCND:US1NJES0015: 2025-03-01 — 2025-03-31
🔎 Скачиваем GHCND:US1NJES0015: 2025-04-01 — 2025-04-30
🔎 Скачиваем GHCND:US1NJES0015: 2025-05-01 — 2025-05-31
🔎 Скачиваем GHCND:US1NJES0015: 2025-06-01 — 2025-06-30
🔎 Скачиваем GHCND:US1NJES0015: 2025-07-01 — 2025-07-31
🔎 Скачиваем GHCND:US1NJES0015: 2025-08-01 — 2025-08-31
🔎 Скачиваем GHCND:US1NJES0015: 2025-09-01 — 2025-09-30
🔎 Скачиваем GHCND:US1NJES0015: 2025-10-01 — 2025-10-31
🔎 Скачиваем GHCND:US1NJES0015: 2025-11-01 — 2025-11-30
🔎 Скачиваем GHCND:US1NJES0015: 2025-12-01 — 2025-12-31
🔎 Скачивае

KeyboardInterrupt: 

In [21]:
import pandas as pd

# --------------------------------------
# 1. Загружаем погодный датасет
# --------------------------------------

# Путь к твоему CSV с погодой
weather_path = "nyc_weather_2025.csv"

# Загружаем
df_weather = pd.read_csv(weather_path, parse_dates=["date"])

# Оставляем только Центральный парк
df_weather_citywide = df_weather[df_weather["station"] == "GHCND:USW00094728"].copy()
df_weather_citywide["date"] = pd.to_datetime(df_weather_citywide["date"]).dt.date

# Удаляем колонку station (она больше не нужна)
df_weather_citywide.drop(columns=["station"], inplace=True)

# Для проверки
print("✅ Погодный датасет:")
print(df_weather_citywide.head())

# --------------------------------------
# 2. Загружаем такси-датасет
# --------------------------------------

# Путь к твоему parquet-файлу
taxi_path = "green_tripdata_aggregated.csv"

df_taxi = pd.read_csv(taxi_path)

print("✅ Такси-датасет:")
print(df_taxi.head())

# --------------------------------------
# 3. Создаем колонку даты в такси-датасете
# --------------------------------------

# Если в твоем датасете есть pickup_hour:
if "pickup_date" in df_taxi.columns:
    # pickup_date может быть строкой вроде "2025-01-01 15"
    df_taxi["date"] = pd.to_datetime(df_taxi["pickup_date"]).dt.date

elif "pickup_datetime" in df_taxi.columns:
    # Если в датасете сохранился оригинальный datetime
    df_taxi["date"] = pd.to_datetime(df_taxi["pickup_datetime"]).dt.date

else:
    raise ValueError("Не нашел колонку pickup_hour или pickup_datetime в taxi-датасете!")

print("✅ Добавлена колонка date в такси-датасет:")
print(df_taxi[["pickup_hour", "date"]].head())

# --------------------------------------
# 4. Склеиваем два датасета
# --------------------------------------

df_merged = df_taxi.merge(
    df_weather_citywide,
    left_on="date",
    right_on="date",
    how="left"
)

print("✅ Итоговый склеенный датасет:")
print(df_merged.head())

# --------------------------------------
# 5. Сохраняем результат
# --------------------------------------

df_merged.to_parquet("nyc_taxi_weather_merged.parquet", index=False)
# или, если хочешь CSV:
# df_merged.to_csv("nyc_taxi_weather_merged.csv", index=False)

print("✅ Итоговый файл сохранен: nyc_taxi_weather_merged.parquet")

✅ Погодный датасет:
         date  AWND    PGTM  PRCP  SNOW  SNWD  TMAX  TMIN   WDF2   WDF5  WSF2  \
0  2025-01-01   3.7  1606.0   0.0   0.0   0.0  10.6   3.9  280.0  270.0   9.8   
1  2025-01-02   4.3  1508.0   0.0   0.0   0.0   6.1   0.6  280.0  230.0   8.9   
2  2025-01-03   3.5  1439.0   0.0   0.0   0.0   3.9  -0.5  230.0  230.0   6.7   
3  2025-01-04   4.5  1302.0   0.0   0.0   0.0   0.6  -2.1  270.0  320.0   9.8   
4  2025-01-05   3.8  1614.0   0.0   0.0   0.0   0.6  -2.1  310.0  310.0   8.1   

   WSF5  WT01  WT02  WT03  WT06  WT08  
0  16.1   NaN   NaN   NaN   NaN   NaN  
1  17.4   NaN   NaN   NaN   NaN   NaN  
2  12.1   NaN   NaN   NaN   NaN   NaN  
3  17.9   NaN   NaN   NaN   NaN   NaN  
4  13.4   NaN   NaN   NaN   NaN   NaN  
✅ Такси-датасет:
   pickup_location_id pickup_date  pickup_hour  trip_time  \
0                 1.0  2022-01-01            4        NaN   
1                 1.0  2022-01-01            5        NaN   
2                 1.0  2022-01-01            7       

In [23]:
df_merged.columns

Index(['pickup_location_id', 'pickup_date', 'pickup_hour', 'trip_time',
       'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'cbd_congestion_fee', 'VendorID', 'passenger_count', 'trip_distance',
       'RatecodeID', 'payment_type', 'fare_amount', 'extra', 'mta_tax',
       'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
       'Airport_fee', 'trip_type', 'dropoff_location_id', 'ride_count', 'date',
       'AWND', 'PGTM', 'PRCP', 'SNOW', 'SNWD', 'TMAX', 'TMIN', 'WDF2', 'WDF5',
       'WSF2', 'WSF5', 'WT01', 'WT02', 'WT03', 'WT06', 'WT08'],
      dtype='object')

In [24]:
df_merged

,pickup_location_id,pickup_date,pickup_hour,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,...,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT06,WT08
0,1.0,2022-01-01,4,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2022-01-01,5,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,2022-01-01,7,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,2022-01-01,10,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,2022-01-01,11,NaN,NaN,NaN,NaN,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731722,265.0,2025-04-30,19,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,15.0,300.0,300.0,6.7,11.6,NaN,NaN,NaN,NaN,NaN
731723,265.0,2025-04-30,20,1427.0,23.55,0.0,0.585,2.08,0.0,0.0,...,15.0,300.0,300.0,6.7,11.6,NaN,NaN,NaN,NaN,NaN
731724,265.0,2025-04-30,21,1339.0,24.56,0.0,0.550,0.00,0.0,0.0,...,15.0,300.0,300.0,6.7,11.6,NaN,NaN,NaN,NaN,NaN
731725,265.0,2025-04-30,22,NaN,NaN,NaN,NaN,NaN,0.0,NaN,...,15.0,300.0,300.0,6.7,11.6,NaN,NaN,NaN,NaN,NaN


In [25]:
# Поля, которые хотим удалить
columns_to_drop = [
    "tips", 
    "tip_amount", 
    "airport_fee", 
    "Airport_fee",
    "tolls", 
    "tolls_amount", 
    "mta_tax",
    "bcf", 
    "improvement_surcharge", 
    "base_passenger_fare",
    "VendorID", 
    "trip_type", 
    "date", 
    "PGTM", 
    "WDF2", 
    "WDF5", 
    "WSF2", 
    "WSF5", 
    "WT02", 
    "WT06", 
    "WT08"
]

# Удаляем только те столбцы, которые реально есть в датафрейме
cols_existing = [col for col in columns_to_drop if col in df_taxi.columns]

print("Будут удалены столбцы:", cols_existing)

# Удаляем
df_taxi_cleaned = df_taxi.drop(columns=cols_existing)

Будут удалены столбцы: ['tips', 'tip_amount', 'airport_fee', 'Airport_fee', 'tolls', 'tolls_amount', 'mta_tax', 'bcf', 'improvement_surcharge', 'base_passenger_fare', 'VendorID', 'trip_type', 'date']


In [26]:
df_taxi_cleaned

,pickup_location_id,pickup_date,pickup_hour,trip_time,sales_tax,congestion_surcharge,driver_pay,cbd_congestion_fee,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,total_amount,dropoff_location_id,ride_count
0,1.0,2022-01-01,4,NaN,NaN,0.0,NaN,NaN,3.000000,0.000000,5.000000,1.000000,40.000000,0.000000,40.300,1.000000,1
1,1.0,2022-01-01,5,NaN,NaN,0.0,NaN,NaN,2.000000,0.000000,5.000000,1.000000,95.000000,0.000000,105.300,1.000000,1
2,1.0,2022-01-01,7,NaN,NaN,0.0,NaN,NaN,2.000000,0.000000,5.000000,1.000000,96.500000,0.000000,107.830,1.000000,2
3,1.0,2022-01-01,10,NaN,NaN,0.0,NaN,NaN,2.000000,0.000000,5.000000,1.000000,79.000000,0.000000,98.050,1.000000,1
4,1.0,2022-01-01,11,NaN,NaN,0.0,NaN,NaN,1.500000,0.000000,5.000000,1.000000,102.500000,0.000000,105.300,1.000000,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731722,265.0,2025-04-30,19,NaN,NaN,0.0,NaN,0.0,1.666667,0.120000,5.000000,1.000000,119.396667,0.000000,139.260,217.500000,4
731723,265.0,2025-04-30,20,1427.0,2.08,0.0,22.705,0.0,1.000000,4.844000,1.800000,3.400000,12.000000,0.000000,15.012,248.333333,9
731724,265.0,2025-04-30,21,1339.0,0.00,0.0,37.180,0.0,1.000000,1.667500,5.000000,1.000000,95.750000,0.000000,113.365,255.000000,6
731725,265.0,2025-04-30,22,NaN,NaN,0.0,NaN,0.0,0.666667,1.673333,3.333333,1.333333,31.266667,0.666667,60.000,197.833333,6


In [27]:
df_taxi_cleaned.to_parquet("nyc_taxi_weather_merged.parquet", index=False)